In [29]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import time
import datetime as dt
import pandas as pd
import numpy as np
import json
import requests
import csv
import os

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'XXXXXXXX'
        consumer_secret = 'XXXXXXXX'
        access_token = 'XXXXXXXX-XXXXXXXX'
        access_token_secret = 'XXXXXXXXXXXX'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))


# Main function for scraping twitter
main function queries twitter at the current time and retrieves the 
sentiment from each tweet retrieved from query

In [2]:
 def main(topic, querySize, summary = True):
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    if type(topic) == str:
        tweets = api.get_tweets(query = topic, count = querySize)
    else:
        tweets = []
        for t in topic:
            tweets = tweets + api.get_tweets(query = t, count = querySize)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    
    # number of neutral tweets
    neutralCount = len(tweets) - len(ntweets) - len(ptweets)
    
    if summary:
        # percentage of positive tweets
        print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
        # percentage of negative tweets
        print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
        # percentage of neutral tweets
        print("Neutral tweets percentage: {} % ".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))

        # printing first 5 positive tweets
        print("\n\nPositive tweets:")
        for tweet in ptweets[:5]:
            print(tweet['text'])

        # printing first 5 negative tweets
        print("\n\nNegative tweets:")
        for tweet in ntweets[:5]:
            print(tweet['text'])
            
    return {'positive': len(ptweets), 
            'negative': len(ntweets), 
            'neutral' : neutralCount}
 
    #if __name__ == "__main__":
        # calling main function
    #    main()

# Creating a dataframe and updated csv for twitter feed and gdax BTC price

In [79]:
def save_data(topic, n):
    curr_sentiment = main(topic, n, False)
    response = requests.get('https://api.cryptowat.ch/markets/gdax/btcusd/summary')
    json_data = json.loads(response.text)
    btcdata = json_data['result']['price']
    btcdata['volume'] = json_data['result']['volume']
    del btcdata['change']

    curr_sentiment = {**curr_sentiment, **btcdata}

    df = pd.DataFrame(curr_sentiment, index = [dt.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')])
    
    return df

In [ ]:
root = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Crypto Algo\\rawcsv\\'
i = 0
starttime = time.time()
mainfile = 'btcdata.csv'

topic = ['BTC','btc','bitcoin','Bitcoin','BitCoin']
n = 1000000

while True:
    df = save_data(topic, n)
    if os.path.isfile(mainfile) == True:
        df.to_csv(mainfile, mode = 'a', header = False)
    else:
        df.to_csv(mainfile)
    df.to_csv(root + 'df' + str(i) + '.csv')    
    time.sleep(60 - ((time.time() - starttime) % 60))

In [ ]:
# import threading

# def hello_world():
#  threading.Timer(5.0, hello_world).start() # called every minute
#  print("Hello, World!")

# hello_world()

#def f(f_stop):
    # do something here ...
#    if not f_stop.is_set():
        # call f() again in 60 seconds
#        threading.Timer(60, f, [f_stop]).start()

#f_stop = threading.Event()
# start calling f now and every 60 sec thereafter
#f(f_stop)

# stop the thread when needed
#f_stop.set()